Load CSV file into memory

In [1]:
import pandas as pd
import requests
import json
import time
from pandas.io.json import json_normalize
 
df = pd.read_csv('Global_country_populations_2013.csv') 
df = df[['CountryName','lat','lon']]
df.head()

,CountryName,lat,lon
0,Aruba,12.52368,-70.03487
1,Andorra,42.55920,1.57020
2,Afghanistan,34.53156,69.12535
3,Angola,-12.33290,16.83090
4,Albania,41.32233,19.82516
